En este archivo voy a explicar cómo desarrollé y creé el primer sistema de recomendación item-item.
Este sistema de recomendación funciona de la siguiente manera:

![Sistema de recomendación item based](../img/item-based.jpg)

De forma muy breve y resumida, este algoritmo recomienda otros items (en nuestro caso juegos) basado en las categorías de los mismos. 

Se me pide lo siguiente:

![Consigna sistema de recomendación](../img/sistema_recomendacion_consigna_item_item.png)

Importación de las librerías necesarias para trabajar.

In [1]:
# Pandas para leer el archivo
import pandas as pd

# Scikit-learn para el sistema de recomendación
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Lectura y vistazo al archivo de juegos con el que voy a desarrollar el sistema de recomendación
juegos_steam = pd.read_parquet("../Datasets/steam_games_complete.parquet") 
juegos_steam.head(1)

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99


Veamos las columnas que tengo disponibles en este archivo

In [3]:
juegos_steam.columns

Index(['item_id', 'item_name', 'developer', 'genres', 'tags', 'specs',
       'release_date', 'price'],
      dtype='object')

De todas estas columnas, las que me parecen más relevantes para este sistema de recomendación son las siguientes:

- 'item_id': Es lo que ingresaré a la función del sistema de recomendación
- 'item_name': Es lo que me va a devolver la función del sistema de recomendación
- 'genres': Es en lo que se va a basar el sistema de recomendación para recomendar.
- 'tags': Es en lo que se va a basar el sistema de recomendación para recomendar.
- 'specs': Es en lo que se va a basar el sistema de recomendación para recomendar.


In [4]:
# Dejamos en el dataframe sólo las columnas útiles y vemos el resultado
juegos_steam = juegos_steam[["item_id","item_name","genres","tags","specs"]]
juegos_steam.head(3)

,item_id,item_name,genres,tags,specs
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player]
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla..."
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla..."


Mi idea de conservar las columnas "genres", "tags" y "specs" es en realidad unirlas en una única columna que contenga toda la información de las mismas para que el sistema de recomendación se base únicamente en esta última columna para hacer sus respectivas recomendaciones.

Pruebo con el método extend

In [5]:
# Tomemos cómo referencia la primera fila, cuyo indice es 88310
generos = list(juegos_steam["genres"][88310])
generos

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

In [6]:
# Ahora le quiero agregar las etiquetas 
etiquetas = list(juegos_steam["tags"][88310])
etiquetas

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']

In [7]:
especificaciones = list(juegos_steam["specs"][88310])
especificaciones

['Single-player']

Probamos uniendo estas 3 listas

In [8]:
generos.extend(etiquetas)
generos.extend(especificaciones)
print(generos)

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy', 'Strategy', 'Action', 'Indie', 'Casual', 'Simulation', 'Single-player']


Bueno ahora voy a tratar de escalar lo que acabo de hacer con un bucle que itere todas las filas del Dataframe

In [9]:
# Lo que hago en este bloque de código es pasar las columnas genres, tags y specs a un diccionario que tiene cómo clave el indice del dataframe y cómo valor una lista que contiene los atributos de las listas contenidas en esas 3 columnas.

diccionario_categorico = {} # Creo un diccionario que va almacenar los datos en este formato: "Indice:Lista con descripción del juego"

for indice in juegos_steam.index: # Se itera sobre el indice del dataframe 

    lista_categorica = [] # Lista que en cada iteración va a almacenar los valores correspondientes a la descripción del juego

    # Agregamos a la lista los valores correspondientes a la fila que se está iterando en la columna genres, tags y specs
    lista_categorica.extend(list(juegos_steam["genres"][indice]))
    lista_categorica.extend(list(juegos_steam["tags"][indice]))
    lista_categorica.extend(list(juegos_steam["specs"][indice]))
    
    # Almacenamos en el diccionario la clave correspondiente al indice y el valor correspondiente a la lista previamente creada.
    diccionario_categorico[indice] = lista_categorica
    
# Veamos cómo quedo el resultado final
diccionario_categorico

{88310: ['Action',
  'Casual',
  'Indie',
  'Simulation',
  'Strategy',
  'Strategy',
  'Action',
  'Indie',
  'Casual',
  'Simulation',
  'Single-player'],
 88311: ['Free to Play',
  'Indie',
  'RPG',
  'Strategy',
  'Free to Play',
  'Strategy',
  'Indie',
  'RPG',
  'Card Game',
  'Trading Card Game',
  'Turn-Based',
  'Fantasy',
  'Tactical',
  'Dark Fantasy',
  'Board Game',
  'PvP',
  '2D',
  'Competitive',
  'Replay Value',
  'Character Customization',
  'Female Protagonist',
  'Difficult',
  'Design & Illustration',
  'Single-player',
  'Multi-player',
  'Online Multi-Player',
  'Cross-Platform Multiplayer',
  'Steam Achievements',
  'Steam Trading Cards',
  'In-App Purchases'],
 88312: ['Casual',
  'Free to Play',
  'Indie',
  'Simulation',
  'Sports',
  'Free to Play',
  'Simulation',
  'Sports',
  'Casual',
  'Indie',
  'Multiplayer',
  'Single-player',
  'Multi-player',
  'Online Multi-Player',
  'In-App Purchases',
  'Stats'],
 88313: ['Action',
  'Adventure',
  'Casual',


Bueno, ahora que ya tengo un diccionario de la forma {Indice:["Categoria A", "Categoria B" ... etc]}, si notamos lo que sucede en nuestro primer par clave:valor, vemos lo siguiente: 

In [10]:
diccionario_categorico[88310]

['Action',
 'Casual',
 'Indie',
 'Simulation',
 'Strategy',
 'Strategy',
 'Action',
 'Indie',
 'Casual',
 'Simulation',
 'Single-player']

Vemos que por ejemplo Strategy se repite, y no sucede sólo con este indice, sino que sucede con la mayoría, ya que la columna "tags" tenía en general datos muy similares o identicos a la columna "generes", por lo cuál, lo que voy a hacer a continuación es dejar los valores únicos dentro de cada lista.

In [11]:
# Lo que hago con este bucle es interar todo el diccionario previamente creado
for indice in diccionario_categorico:
    # En cada iteración, cambia los valores: "diccionario_categorico[i]" para dejar unicamente los datos dentro de cada lista que no se repiten con el método "dict.fromkeys" y lo deja en formato lista con el método list
    diccionario_categorico[indice] = list(dict.fromkeys(diccionario_categorico[indice]))
    print(diccionario_categorico[indice]) # Vemos que con el primer indice funciona correctamente, así que lo dejo iterar sobre todo el diccionario
    break


['Action', 'Casual', 'Indie', 'Simulation', 'Strategy', 'Single-player']


In [12]:
# Lo que hago con este bucle es interar todo el diccionario previamente creado
for indice in diccionario_categorico:
    # En cada iteración, cambia los valores: "diccionario_categorico[i]" para dejar unicamente los datos dentro de cada lista que no se repiten con el método "dict.fromkeys" y lo deja en formato lista con el método list
    diccionario_categorico[indice] = list(dict.fromkeys(diccionario_categorico[indice]))

# Finalmente veamos cómo queda el diccionario
diccionario_categorico

{88310: ['Action',
  'Casual',
  'Indie',
  'Simulation',
  'Strategy',
  'Single-player'],
 88311: ['Free to Play',
  'Indie',
  'RPG',
  'Strategy',
  'Card Game',
  'Trading Card Game',
  'Turn-Based',
  'Fantasy',
  'Tactical',
  'Dark Fantasy',
  'Board Game',
  'PvP',
  '2D',
  'Competitive',
  'Replay Value',
  'Character Customization',
  'Female Protagonist',
  'Difficult',
  'Design & Illustration',
  'Single-player',
  'Multi-player',
  'Online Multi-Player',
  'Cross-Platform Multiplayer',
  'Steam Achievements',
  'Steam Trading Cards',
  'In-App Purchases'],
 88312: ['Casual',
  'Free to Play',
  'Indie',
  'Simulation',
  'Sports',
  'Multiplayer',
  'Single-player',
  'Multi-player',
  'Online Multi-Player',
  'In-App Purchases',
  'Stats'],
 88313: ['Action', 'Adventure', 'Casual', 'Single-player'],
 88315: ['Action',
  'Adventure',
  'Simulation',
  'FPS',
  'Shooter',
  'Third-Person Shooter',
  'Sniper',
  'Third Person',
  'Single-player',
  'Steam Achievements'],


Ahora que ya tengo el diccionario, lo que voy a hacer es agregar una nueva columna al dataframe de juegos que contenga los valores correspondientes.

In [13]:
# Agrego la columna "categorical" que va a ser la combinación de "genres", "tags", "specs" con sus valores únicos y lo muetro en pantalla
juegos_steam['categorical'] = juegos_steam.index.map(diccionario_categorico)
juegos_steam.head(1)

,item_id,item_name,genres,tags,specs,categorical
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],"[Action, Casual, Indie, Simulation, Strategy, ..."


Ahora, cómo en la columna categorical tengo los valores únicos correspondientes a las columnas "genres","tags" y "specs", estas 3 columnas resultan redundantes, por lo cuál decido eliminarlas dejando únicamente:

- item_id: Lo que se ingresa a la función
- Item_name: Lo que devuelve la función
- Categorical: Lo que usa la función para recomendar

De esta manera, optimizo el rendimiento de la función y del sistema de recomendación

In [14]:
# Para el sistema de recomendación, las únicas 3 columnas que necesito son las que dejo en el Dataframe
juegos_steam = juegos_steam[["item_id","item_name","categorical"]]
juegos_steam # Echemos un vistazo a cómo me quedo el Dataframe luego de los cambios realizados

,item_id,item_name,categorical
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy, ..."
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy, Card Game..."
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Spor..."
88313,767400,弹炸人2222,"[Action, Adventure, Casual, Single-player]"
88315,772540,Battle Royale Trainer,"[Action, Adventure, Simulation, FPS, Shooter, ..."
...,...,...,...
120439,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie, Violent, Si..."
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy, Single-p..."
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy, Single-player, Steam..."
120442,610660,Russian Roads,"[Indie, Racing, Simulation, Single-player, Ste..."


In [15]:
# Creo una columna nueva dentro del Dataframe que contiene las listas de cada fila de la columna categorical a un string de la forma: "Genero 1 Genero 2 Etiqueta 1 Etiqueta 2 ..."
juegos_steam["categorical_string"] = juegos_steam["categorical"].apply(lambda x: " ".join(x))
juegos_steam["categorical_string"] # Muestro los resultados de la nueva columna creada

88310     Action Casual Indie Simulation Strategy Single...
88311     Free to Play Indie RPG Strategy Card Game Trad...
88312     Casual Free to Play Indie Simulation Sports Mu...
88313                 Action Adventure Casual Single-player
88315     Action Adventure Simulation FPS Shooter Third-...
                                ...                        
120439    Action Adventure Casual Indie Violent Single-p...
120440    Casual Indie Simulation Strategy Single-player...
120441    Casual Indie Strategy Single-player Steam Achi...
120442    Indie Racing Simulation Single-player Steam Ac...
120443    Casual Indie Puzzle Singleplayer Atmospheric R...
Name: categorical_string, Length: 22530, dtype: object

In [16]:
juegos_steam # Echemos un vistazo a cómo está quedando nuestro dataframe

,item_id,item_name,categorical,categorical_string
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy, ...",Action Casual Indie Simulation Strategy Single...
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy, Card Game...",Free to Play Indie RPG Strategy Card Game Trad...
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Spor...",Casual Free to Play Indie Simulation Sports Mu...
88313,767400,弹炸人2222,"[Action, Adventure, Casual, Single-player]",Action Adventure Casual Single-player
88315,772540,Battle Royale Trainer,"[Action, Adventure, Simulation, FPS, Shooter, ...",Action Adventure Simulation FPS Shooter Third-...
...,...,...,...,...
120439,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie, Violent, Si...",Action Adventure Casual Indie Violent Single-p...
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy, Single-p...",Casual Indie Simulation Strategy Single-player...
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy, Single-player, Steam...",Casual Indie Strategy Single-player Steam Achi...
120442,610660,Russian Roads,"[Indie, Racing, Simulation, Single-player, Ste...",Indie Racing Simulation Single-player Steam Ac...


In [17]:
# Reinicio el indice, ya que al momento de implementar el sistema de recomendación es una de las cosas que utiliza.
juegos_steam.reset_index(drop=True, inplace=True)
juegos_steam.head(1)

,item_id,item_name,categorical,categorical_string
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy, ...",Action Casual Indie Simulation Strategy Single...


In [18]:
# Elimino la columna categorical que creé originalmente, ya que no la voy a utilizar dentro del sistema de recomendación
juegos_steam.drop("categorical", axis=1,inplace=True)
juegos_steam # Vemos los resultados

,item_id,item_name,categorical_string
0,761140,Lost Summoner Kitty,Action Casual Indie Simulation Strategy Single...
1,643980,Ironbound,Free to Play Indie RPG Strategy Card Game Trad...
2,670290,Real Pool 3D - Poolians,Casual Free to Play Indie Simulation Sports Mu...
3,767400,弹炸人2222,Action Adventure Casual Single-player
4,772540,Battle Royale Trainer,Action Adventure Simulation FPS Shooter Third-...
...,...,...,...
22525,745400,Kebab it Up!,Action Adventure Casual Indie Violent Single-p...
22526,773640,Colony On Mars,Casual Indie Simulation Strategy Single-player...
22527,733530,LOGistICAL: South Africa,Casual Indie Strategy Single-player Steam Achi...
22528,610660,Russian Roads,Indie Racing Simulation Single-player Steam Ac...


Finalmente, esta tabla de 22530 filas y 3 columnas va a ser lo que use el sistema de recomendación, por lo cuál, voy a guardarla en formato parquet para poder traerla fácilmente desde el archivo principal (main.py) cuando implemente y use la función allí.

In [19]:
# Exporto el dataframe cómo parquet para poder usarlo en el archivo main.py
juegos_steam.to_parquet("../Datasets/recomendacion/steam_games_id_name_categorical.parquet",compression="snappy")

A continuación se utiliza CountVectorizer, una herramienta del módulo Scikit-learn. La utilizamos para convertir una colección de texto, en este caso, cada fila de nuestra columna "categorical_string" en una matriz de tokens (palabras) con sus respectivas frecuencias. 
Por ejemplo:

In [20]:
cv = CountVectorizer(max_features=21, stop_words='english') # Max features = 21 significa que solo se tendrán en cuenta las 21 palabras más frecuentes de todos los textos combinados.
# Stop words = english filtra y elimina las palabras comunes en inglés que no aportan mucho significado (como "and", "the", "is", etc.), para centrarse en las palabras más relevantes.

Aplicamos el count vectorizer a la columna categorical string creada previamente.

In [21]:
vector = cv.fit_transform(juegos_steam["categorical_string"]).toarray()
#  Este método ajusta el CountVectorizer a los datos (aprende las palabras y sus frecuencias) y luego transforma los textos en una matriz donde cada fila representa una fila y cada columna representa una palabra (de las 21 más comunes). El valor en la celda es el número de veces que aparece la palabra en esa fila.

In [22]:
vector.shape # Vemos que la longitud de filas del vector coincide con la longitud de las filas del Dataframe

(22530, 21)

Echemos un vistazo a ver cómo es este vector.

In [23]:
column_names = cv.get_feature_names_out()
Data = pd.DataFrame(vector, columns=column_names)
Data

,achievements,action,adventure,cards,casual,cloud,content,controller,downloadable,indie,...,multiplayer,op,player,rpg,simulation,single,steam,strategy,support,trading
0,0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,1,1,0,1,0,0
1,1,0,0,1,0,0,0,0,0,1,...,1,0,3,1,0,1,2,1,0,2
2,0,0,0,0,1,0,0,0,0,1,...,1,0,3,0,1,1,0,0,0,0
3,0,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22525,1,1,1,0,1,1,0,0,0,1,...,0,0,1,0,0,1,2,0,0,0
22526,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,1,1,1,0,0
22527,1,0,0,0,1,1,0,0,0,1,...,0,0,1,0,0,1,3,1,0,0
22528,1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,1,1,2,0,0,1


Aplicamos el algoritmo de la similitud del coseno entre los vectores de la matriz que generamos anteriormente. La similitud del coseno es una medida que indica cómo de similares son dos documentos en términos de su contenido textual. Devuelve una matriz donde cada elemento (i, j) representa la similitud del coseno entre los documentos i y j.

In [24]:
similitud = cosine_similarity(vector)
similitud.shape

(22530, 22530)

Almacenamos este vector en un DataFrame y lo exportamos en formato .parquet para utilizarlo posteriormente en el main.py

In [25]:
similitud_entre_juegos = pd.DataFrame(similitud)
similitud_entre_juegos

,0,1,2,3,4,5,6,7,8,9,...,22520,22521,22522,22523,22524,22525,22526,22527,22528,22529
0,1.000000,0.428571,0.623610,0.676123,0.571429,0.354169,0.354169,0.629941,0.755929,0.845154,...,0.654654,0.654654,0.462910,0.455842,0.377964,0.545545,0.801784,0.472456,0.455842,0.478091
1,0.428571,1.000000,0.712697,0.338062,0.500000,0.826394,0.826394,0.566947,0.472456,0.422577,...,0.600099,0.545545,0.617213,0.569803,0.692935,0.545545,0.601338,0.614192,0.740744,0.597614
2,0.623610,0.712697,1.000000,0.527046,0.445435,0.625779,0.625779,0.471405,0.707107,0.737865,...,0.748455,0.680414,0.481125,0.426401,0.392837,0.408248,0.583333,0.353553,0.426401,0.447214
3,0.676123,0.338062,0.527046,1.000000,0.676123,0.279372,0.279372,0.745356,0.670820,0.600000,...,0.645497,0.516398,0.547723,0.404520,0.298142,0.645497,0.474342,0.335410,0.269680,0.424264
4,0.571429,0.500000,0.445435,0.676123,1.000000,0.590281,0.590281,0.755929,0.377964,0.507093,...,0.654654,0.436436,0.771517,0.569803,0.629941,0.763763,0.668153,0.566947,0.683763,0.597614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22525,0.545545,0.545545,0.408248,0.645497,0.763763,0.676252,0.676252,0.866025,0.577350,0.516398,...,0.416667,0.500000,0.824958,0.783349,0.769800,1.000000,0.714435,0.866025,0.696311,0.912871
22526,0.801784,0.601338,0.583333,0.474342,0.668153,0.607373,0.607373,0.707107,0.707107,0.790569,...,0.510310,0.612372,0.721688,0.746203,0.707107,0.714435,1.000000,0.795495,0.746203,0.782624
22527,0.472456,0.614192,0.353553,0.335410,0.566947,0.741825,0.741825,0.666667,0.500000,0.447214,...,0.288675,0.433013,0.714435,0.829156,0.833333,0.866025,0.795495,1.000000,0.753778,0.948683
22528,0.455842,0.740744,0.426401,0.269680,0.683763,0.800499,0.800499,0.603023,0.452267,0.539360,...,0.348155,0.522233,0.738549,0.727273,0.804030,0.696311,0.746203,0.753778,1.000000,0.762770


In [26]:
# similitud_entre_juegos.to_parquet("../Datasets/recomendacion/similitud_entre_juegos.parquet",compression="snappy")

Me acabo de dar cuenta que el bloque anterior de código crea un archivo .parquet que pesa 700 mb aproximadamente, lo cuál es una barbaridad a nivel computo, sobre todo para desplegar en render, entonces la estrategia que voy a usar es la siguiente:
22530 filas x 22530 columnas nos dá un total de 507.600.900 celdas.
Si 507.600.900 celdas pesan casí 700mb (decimos 700 para redondear).
Entonces X celdas pesan 50 mb, lo cuál es lo máximo que estoy dispuesto a tolerar por tabla.
X = (50 mb * 507.600.900 celdas) / 700 mb = 36.257.207,14 celdas
Si aplicamos la raíz cuadrada a 36.257.207,14 para obtener la cantidad de filas y columnas, nos daría cómo resultado nuevos dataframes de 6021 filas x 6021 columnas.


Entonces lo que voy a hacer es lo siguiente, voy a partir el dataframe original en 4 partes. Las 3 primeras de 6021 columnas y la última de 4467 columnas.


In [ ]:
# # Definir el número de columnas en cada parte
# columnas_por_parte = 6021

# # Calculamos la cantidad de partes
# cantidad_partes = similitud_entre_juegos.shape[1] // columnas_por_parte + (1 if similitud_entre_juegos.shape[1] % columnas_por_parte != 0 else 0) # En total son 4 partes, 3 de 6021 columnas y la última de 4467

# # Por cada parte
# for parte in range(cantidad_partes):

#     # Definimos el rango de columnas para esta parte
#     start_col = parte * columnas_por_parte 
#     end_col = start_col + columnas_por_parte
    
#     # Seleccionar la parte del DataFrame
#     df_part = similitud_entre_juegos.iloc[:, start_col:end_col]
    
#     # Exportar esta parte a un archivo CSV
#     df_part.to_csv(f"df_part_{parte+1}.csv", index=False)
    
#     print(f"Parte {parte+1} exportada: {start_col} a {end_col - 1} columnas")


Bueno, la regla de 3 simples anteriormente mencionada, no fué muy acertada jajaja. Me quedaron 4 archivos de 200 mb cada uno. Así que voy a probar con menos columnas a ver que tal.

Pruebo con 1500 columnas (aproximadamente 1/4) de lo anteriormente probado. 

In [28]:
df_part = similitud_entre_juegos.iloc[:, 0:1500]
df_part

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,1.000000,0.428571,0.623610,0.676123,0.571429,0.354169,0.354169,0.629941,0.755929,0.845154,...,0.421117,0.188982,0.415339,0.419314,0.422577,0.236113,0.356348,0.341882,0.545545,0.341882
1,0.428571,1.000000,0.712697,0.338062,0.500000,0.826394,0.826394,0.566947,0.472456,0.422577,...,0.736956,0.566947,0.778761,0.524142,0.676123,0.678824,0.668153,0.398862,0.872872,0.683763
2,0.623610,0.712697,1.000000,0.527046,0.445435,0.625779,0.625779,0.471405,0.707107,0.737865,...,0.525226,0.196419,0.647524,0.392232,0.316228,0.331295,0.333333,0.284268,0.748455,0.355335
3,0.676123,0.338062,0.527046,1.000000,0.676123,0.279372,0.279372,0.745356,0.670820,0.600000,...,0.332182,0.149071,0.430007,0.496139,0.200000,0.279372,0.316228,0.674200,0.387298,0.404520
4,0.571429,0.500000,0.445435,0.676123,1.000000,0.590281,0.590281,0.755929,0.377964,0.507093,...,0.561490,0.566947,0.623009,0.628971,0.591608,0.590281,0.534522,0.797724,0.545545,0.569803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22525,0.545545,0.545545,0.408248,0.645497,0.763763,0.676252,0.676252,0.866025,0.577350,0.516398,...,0.643268,0.673575,0.713746,0.880705,0.710047,0.721336,0.748455,0.696311,0.416667,0.783349
22526,0.801784,0.601338,0.583333,0.474342,0.668153,0.607373,0.607373,0.707107,0.707107,0.790569,...,0.656532,0.530330,0.582772,0.686406,0.711512,0.496942,0.666667,0.426401,0.612372,0.533002
22527,0.472456,0.614192,0.353553,0.335410,0.566947,0.741825,0.741825,0.666667,0.500000,0.447214,...,0.742781,0.791667,0.686803,0.832050,0.838525,0.780869,0.824958,0.452267,0.505181,0.753778
22528,0.455842,0.740744,0.426401,0.269680,0.683763,0.800499,0.800499,0.603023,0.452267,0.539360,...,0.727860,0.804030,0.704068,0.668994,0.876460,0.753411,0.852803,0.454545,0.696311,0.818182


In [29]:
# df_part.to_parquet("probando_espacio.parquet",compression="snappy") 31mb, voy a probar con más columnas

In [30]:
# df_part = similitud_entre_juegos.iloc[:, 0:1800] 56mb , me parece óptimo
# df_part.to_parquet("probando_espacio.parquet",compression="snappy")

Bueno, pruebo con 1800 columnas

In [31]:
# Definir el número de columnas en cada parte
columnas_por_parte = 1800

# Calculamos la cantidad de partes
cantidad_partes = similitud_entre_juegos.shape[1] // columnas_por_parte + (1 if similitud_entre_juegos.shape[1] % columnas_por_parte != 0 else 0) 

# Por cada parte
for parte in range(cantidad_partes):

    # Definimos el rango de columnas para esta parte
    start_col = parte * columnas_por_parte 
    end_col = start_col + columnas_por_parte
    
    # Seleccionar la parte del DataFrame
    df_part = similitud_entre_juegos.iloc[:, start_col:end_col]
    
    # Exportar esta parte a un archivo parquet
    df_part.to_parquet(f"../Datasets/recomendacion/uno/similitud_entre_juegos{parte+1}.parquet", index=False, compression="snappy")
    
    print(f"Parte {parte+1} exportada: {start_col} a {end_col - 1} columnas")


Parte 1 exportada: 0 a 1799 columnas
Parte 2 exportada: 1800 a 3599 columnas
Parte 3 exportada: 3600 a 5399 columnas
Parte 4 exportada: 5400 a 7199 columnas
Parte 5 exportada: 7200 a 8999 columnas
Parte 6 exportada: 9000 a 10799 columnas
Parte 7 exportada: 10800 a 12599 columnas
Parte 8 exportada: 12600 a 14399 columnas
Parte 9 exportada: 14400 a 16199 columnas
Parte 10 exportada: 16200 a 17999 columnas
Parte 11 exportada: 18000 a 19799 columnas
Parte 12 exportada: 19800 a 21599 columnas
Parte 13 exportada: 21600 a 23399 columnas


Primer prototipo del sistema de recomendación

In [112]:
# Función que toma cómo argumento el ID de un juego y recomienda 5 similares
def recomendacion_juego(id_juego):
    """Implementación de los archivos"""
    # Busca en 
    indice_juego = juegos_steam[juegos_steam["item_id"] == id_juego].index[0]

    if indice_juego <= 1799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos1.parquet")
    elif indice_juego <= 3599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos2.parquet")
    elif indice_juego <= 5399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos3.parquet")
    elif indice_juego <= 7199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos4.parquet")
    elif indice_juego <= 8999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos5.parquet")
    elif indice_juego <= 10799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos6.parquet")
    elif indice_juego <= 12599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos7.parquet")
    elif indice_juego <= 14399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos8.parquet")
    elif indice_juego <= 16199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos9.parquet")
    elif indice_juego <= 17999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos10.parquet")
    elif indice_juego <= 19799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos11.parquet")
    elif indice_juego <= 21599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos12.parquet")
    elif indice_juego <= 22529:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos13.parquet")

    # distancias = sorted(list(enumerate(similitud[indice_juego])), reverse=True, key=lambda x: x[1])
    
    # juegos_recomendados = []
    # detalles_juego = []
    # salida = {}

    # for i in distancias[1:6]:
    #     juegos_recomendados.append(juegos_steam.iloc[i[0]].item_name)
    #     detalles_juego.append(juegos_steam.iloc[i[0]].categorical)
    
    # for i,j in enumerate(juegos_recomendados):
    #     salida[j] = detalles_juego[i]

    return salida
    

        
    

Segundo prototipo del endpoint

In [52]:
# Función que toma cómo argumento el ID de un juego y recomienda 5 similares
def recomendacion_juego(id_juego):
    """Implementación de los archivos"""

    # Definimos una variable de salida 
    salida = {}

    # Encontramos el nombre del juego y lo guardamos en el diccionario de salida
    nombre_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["item_name"].iloc[0]
    salida["Nombre del juego ingresado"] = nombre_juego

    # Encontramos las categorías del juego ingresado y lo guardamos en el diccionario de salida
    categorias_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["categorical_string"].iloc[0]
    salida["Características del juego ingresado"] = categorias_juego

    # Buscamos el indice del juego
    indice_juego = juegos_steam[juegos_steam["item_id"] == id_juego].index[0]

    # Según el indice del juego, cargamos uno u otro datasets. Cada dataset contiene 22530 filas y 1800 columnas, excepto el último que contiene 22530 filas y 930 columnas.
    if indice_juego <= 1799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos1.parquet")
    elif indice_juego <= 3599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos2.parquet")
    elif indice_juego <= 5399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos3.parquet")
    elif indice_juego <= 7199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos4.parquet")
    elif indice_juego <= 8999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos5.parquet")
    elif indice_juego <= 10799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos6.parquet")
    elif indice_juego <= 12599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos7.parquet")
    elif indice_juego <= 14399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos8.parquet")
    elif indice_juego <= 16199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos9.parquet")
    elif indice_juego <= 17999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos10.parquet")
    elif indice_juego <= 19799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos11.parquet")
    elif indice_juego <= 21599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos12.parquet")
    elif indice_juego <= 22529:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos13.parquet")



    distancias = sorted(list(enumerate(similitud_entre_juegos.loc[indice_juego])), reverse=True, key=lambda x: x[1])
    
    juegos_recomendados = []
    detalles_juego = []
    

    # for i in distancias[1:6]:
    #     juegos_recomendados.append(juegos_steam.iloc[i[0]].item_name)
    #     detalles_juego.append(juegos_steam.iloc[i[0]].categorical)
    
    # for i,j in enumerate(juegos_recomendados):
    #     salida[j] = detalles_juego[i]

    return distancias
    

        
    

A continuación pruebo que la función devuelva un valor esperado

In [53]:
recomendacion_juego("761140")

[(0, 0.9999999999999997),
 (1139, 0.9258200997725515),
 (9, 0.8451542547285164),
 (21, 0.8451542547285164),
 (26, 0.8451542547285164),
 (557, 0.8451542547285164),
 (694, 0.8451542547285164),
 (791, 0.8451542547285164),
 (1043, 0.8451542547285164),
 (1593, 0.8451542547285164),
 (1745, 0.8451542547285164),
 (684, 0.801783725737273),
 (1684, 0.801783725737273),
 (41, 0.7715167498104596),
 (91, 0.7715167498104596),
 (142, 0.7715167498104596),
 (454, 0.7715167498104596),
 (8, 0.7559289460184544),
 (36, 0.7559289460184544),
 (37, 0.7559289460184544),
 (38, 0.7559289460184544),
 (39, 0.7559289460184544),
 (44, 0.7559289460184544),
 (50, 0.7559289460184544),
 (83, 0.7559289460184544),
 (88, 0.7559289460184544),
 (101, 0.7559289460184544),
 (132, 0.7559289460184544),
 (144, 0.7559289460184544),
 (150, 0.7559289460184544),
 (187, 0.7559289460184544),
 (208, 0.7559289460184544),
 (246, 0.7559289460184544),
 (247, 0.7559289460184544),
 (272, 0.7559289460184544),
 (305, 0.7559289460184544),
 (317, 

Tercer prototipo

In [62]:
# Función que toma cómo argumento el ID de un juego y recomienda 5 similares
def recomendacion_juego(id_juego):
    """Implementación de los archivos"""

    # Definimos una variable de salida 
    salida = {}

    # Encontramos el nombre del juego y lo guardamos en el diccionario de salida
    nombre_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["item_name"].iloc[0]
    salida["Nombre del juego ingresado"] = nombre_juego

    # Encontramos las categorías del juego ingresado y lo guardamos en el diccionario de salida
    categorias_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["categorical_string"].iloc[0]
    salida["Características del juego ingresado"] = categorias_juego

    # Buscamos el indice del juego
    indice_juego = juegos_steam[juegos_steam["item_id"] == id_juego].index[0]

    # Según el indice del juego, cargamos uno u otro datasets. Cada dataset contiene 22530 filas y 1800 columnas, excepto el último que contiene 22530 filas y 930 columnas.
    if indice_juego <= 1799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos1.parquet")
    elif indice_juego <= 3599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos2.parquet")
    elif indice_juego <= 5399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos3.parquet")
    elif indice_juego <= 7199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos4.parquet")
    elif indice_juego <= 8999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos5.parquet")
    elif indice_juego <= 10799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos6.parquet")
    elif indice_juego <= 12599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos7.parquet")
    elif indice_juego <= 14399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos8.parquet")
    elif indice_juego <= 16199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos9.parquet")
    elif indice_juego <= 17999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos10.parquet")
    elif indice_juego <= 19799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos11.parquet")
    elif indice_juego <= 21599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos12.parquet")
    elif indice_juego <= 22529:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos13.parquet")


    # Calculamos la distancia entre los juegos (osea, que tan similar es el juego ingresado con los demás) y los ordenamos de más similar a menos similar. Distancias es una lista de tuplas de la forma: [(Indice juego, similitud), (Indice juego, similitud), etc]
    distancias = sorted(list(enumerate(similitud_entre_juegos.loc[indice_juego])), reverse=True, key=lambda x: x[1])
    
    # Creamos una lista que almacena los nombres de los juegos recomendados
    juegos_recomendados = []

    # Creamos una lista que almacena los detalles del juego, es decir las categorías de los mismos
    detalles_juego = []
    
    # Por cada elemento en distancias, desde el 2º (ya que el primero es el mismo juego porque tiene una similitud del 100%) hasta el 6º
    for i in distancias[1:6]:
        juegos_recomendados.append(juegos_steam.iloc[i[0]].item_name)
        detalles_juego.append(juegos_steam.iloc[i[0]].categorical_string)
    
    for i,j in enumerate(juegos_recomendados):
        salida[j] = detalles_juego[i]

    return salida
    

        
    

In [61]:
recomendacion_juego("761140")

{'Nombre del juego ingresado': 'Lost Summoner Kitty',
 'Características del juego ingresado': 'Action Casual Indie Simulation Strategy Single-player',
 'Lost Summoner Kitty': 'Action Casual Indie Simulation Strategy Single-player',
 'Fieldrunners 2': 'Action Strategy Casual Tower Defense Touch-Friendly Indie Singleplayer Single-player',
 'Planetarium 2 - Zen Odyssey': 'Casual Indie Simulation Single-player',
 'Vaporwave Simulator': 'Casual Indie Simulation Single-player',
 'Souland': 'Action Indie Strategy Early Access Pixel Graphics Single-player',
 'Dinner Date': 'Indie Casual Short Experimental Singleplayer First-Person Simulation Narration Story Rich Single-player'}

Ya tengo lo que me piden, ahora antes de probar la función, me gustaría que la salida sea de la forma:

{
    "Nombre del juego ingresado" : X,

    "Características del juego ingresado" : Y,
    
    "Juegos recomendados" : {
        "Juego 1" : Caracteristicas,
        "Juego 2" : Caracteristicas,
        "Juego 3" : Caracteristicas,
        "Juego 4" : Caracteristicas,
        "Juego 5" : Caracteristicas
    }
}

Vamos a ello, cuarto prototipo del endpoint

In [63]:
# Función que toma cómo argumento el ID de un juego y recomienda 5 similares
def recomendacion_juego(id_juego):
    """Implementación de los archivos"""

    # Definimos una variable de salida 
    salida = {}

    # Encontramos el nombre del juego y lo guardamos en el diccionario de salida
    nombre_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["item_name"].iloc[0]
    salida["Nombre del juego ingresado"] = nombre_juego

    # Encontramos las categorías del juego ingresado y lo guardamos en el diccionario de salida
    categorias_juego = juegos_steam[juegos_steam["item_id"] == id_juego]["categorical_string"].iloc[0]
    salida["Características del juego ingresado"] = categorias_juego

    # Buscamos el indice del juego
    indice_juego = juegos_steam[juegos_steam["item_id"] == id_juego].index[0]

    # Según el indice del juego, cargamos uno u otro datasets. Cada dataset contiene 22530 filas y 1800 columnas, excepto el último que contiene 22530 filas y 930 columnas.
    if indice_juego <= 1799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos1.parquet")
    elif indice_juego <= 3599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos2.parquet")
    elif indice_juego <= 5399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos3.parquet")
    elif indice_juego <= 7199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos4.parquet")
    elif indice_juego <= 8999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos5.parquet")
    elif indice_juego <= 10799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos6.parquet")
    elif indice_juego <= 12599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos7.parquet")
    elif indice_juego <= 14399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos8.parquet")
    elif indice_juego <= 16199:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos9.parquet")
    elif indice_juego <= 17999:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos10.parquet")
    elif indice_juego <= 19799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos11.parquet")
    elif indice_juego <= 21599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos12.parquet")
    elif indice_juego <= 22529:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos13.parquet")


    # Calculamos la distancia entre los juegos (osea, que tan similar es el juego ingresado con los demás) y los ordenamos de más similar a menos similar. Distancias es una lista de tuplas de la forma: [(Indice juego, similitud), (Indice juego, similitud), etc]
    distancias = sorted(list(enumerate(similitud_entre_juegos.loc[indice_juego])), reverse=True, key=lambda x: x[1])
    
    # Creamos una lista que almacena los nombres de los juegos recomendados
    juegos_recomendados = []

    # Creamos una lista que almacena los detalles del juego, es decir las categorías de los mismos
    detalles_juego = []

    # Creamos un diccionario interno
    juegos_recomendados_y_caracteristicas = {}
    
    # Por cada elemento en distancias, desde el 2º (ya que el primero es el mismo juego porque tiene una similitud del 100%) hasta el 6º
    for i in distancias[1:6]:
        juegos_recomendados.append(juegos_steam.iloc[i[0]].item_name)
        detalles_juego.append(juegos_steam.iloc[i[0]].categorical_string)
    
    for i,j in enumerate(juegos_recomendados):
        juegos_recomendados_y_caracteristicas[j] = detalles_juego[i]
    
    salida["Juegos recomendados"] = juegos_recomendados_y_caracteristicas

    return salida
    

        
    

In [64]:
recomendacion_juego("761140")

{'Nombre del juego ingresado': 'Lost Summoner Kitty',
 'Características del juego ingresado': 'Action Casual Indie Simulation Strategy Single-player',
 'Juegos recomendados': {'Fieldrunners 2': 'Action Strategy Casual Tower Defense Touch-Friendly Indie Singleplayer Single-player',
  'Planetarium 2 - Zen Odyssey': 'Casual Indie Simulation Single-player',
  'Vaporwave Simulator': 'Casual Indie Simulation Single-player',
  'Souland': 'Action Indie Strategy Early Access Pixel Graphics Single-player',
  'Dinner Date': 'Indie Casual Short Experimental Singleplayer First-Person Simulation Narration Story Rich Single-player'}}

Bueno, con eso doy por finalizada la creación de la función. A continuación lo que voy a hacer en este archivo son algunas pruebas para ver si los archivos .parquet se leen correctamente. Haré una prueba por cada uno de ellos.

Primero veamos los rangos...

indice_juego <= 1799:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos1.parquet")

Sabemos que este funciona correctamente porque es el que estuvimos usando.
Probemos ahora con:
    elif indice_juego <= 3599:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos2.parquet")

Veamos aquellos juegos de indice mayor que 1799, los primeros 1800 que aparezcan son los que queremos probar si funcionan
    

In [68]:
juegos_steam[juegos_steam.index > 1799].head(1800)

,item_id,item_name,categorical_string
1800,271570,Space Farmers,Action Indie Co-op Multiplayer Space Puzzle Lo...
1801,284950,Pixel Puzzles: Japan,Casual Indie Simulation Puzzle Singleplayer Si...
1802,283960,Pajama Sam: No Need to Hide When It's Dark Out...,Adventure Casual Point & Click Family Friendly...
1803,292880,Robin's Quest,Adventure Hidden Object Point & Click Single-p...
1804,283940,Freddi Fish and The Case of the Missing Kelp S...,Adventure Casual Point & Click Family Friendly...
...,...,...,...
3595,389420,Panzer Corps Sea Lion,Strategy Single-player Multi-player Cross-Plat...
3596,243160,Mushroom 11,Action Adventure Indie Puzzle Platformer Puzzl...
3597,408070,Mushroom 11 Soundtrack - The Future Sound of L...,Action Adventure Indie Single-player Downloada...
3598,274190,Broforce,Action Adventure Casual Indie America Pixel Gr...


In [69]:
recomendacion_juego("292880") # La salida es la esperada

{'Nombre del juego ingresado': "Robin's Quest",
 'Características del juego ingresado': 'Adventure Hidden Object Point & Click Single-player',
 'Juegos recomendados': {"Desperados 2: Cooper's Revenge": 'Action Western Strategy Tactical Single-player',
  'X-COM: Enforcer': 'Strategy Shooter Action Third-Person Shooter Sci-fi Aliens Single-player Multi-player',
  'Two Worlds II HD': 'RPG Open World Fantasy Third Person Exploration Singleplayer Multiplayer Co-op Action RPG Crafting Adventure Story Rich Action Hack and Slash Dark Fantasy Great Soundtrack Inventory Management Magic Nudity Violent Single-player Multi-player Online Multi-Player Online Co-op Cross-Platform Multiplayer Steam Achievements Full controller support Steam Trading Cards In-App Purchases Steam Cloud',
  'Warhammer 40,000: Dawn of War II - Retribution Eldar Race Pack': 'Strategy Single-player Multi-player Co-op Downloadable Content Steam Cloud Stats Steam Leaderboards',
  'Cities in Motion: Design Classics': 'Simulatio

Probemos ahora con:
elif indice_juego <= 5399:
        similitud_entre_juegos = pd.read_parquet("../Datasets/recomendacion/uno/similitud_entre_juegos3.parquet")
        
Probamos con todos los mayores a 3599

In [70]:
juegos_steam[juegos_steam.index > 3599].head(1800)

,item_id,item_name,categorical_string
3600,410790,Broforce: The Soundtrack,Action Adventure Casual Indie Single-player Do...
3601,398840,War of Beach,Action Free to Play Simulation Strategy War To...
3602,409420,Knight Adventure,Action Adventure Indie Single-player Steam Tra...
3603,394450,"Warhammer 40,000: Deathwatch - Enhanced Edition",Action Adventure RPG Strategy Warhammer 40K Tu...
3604,263920,Zombie Grinder,Action Adventure Indie RPG Early Access Zombie...
...,...,...,...
5395,476530,Children of a Dead Earth,Simulation Strategy Space Realistic Sci-fi Sci...
5396,521940,Space Incident,Action Adventure Indie Single-player Steam Tra...
5397,527790,Cranium Conundrum,Adventure Casual Indie Early Access Single-pla...
5398,527250,VilleTown,Indie Simulation Strategy Single-player Steam ...


In [71]:
recomendacion_juego("410790")

{'Nombre del juego ingresado': 'Broforce: The Soundtrack',
 'Características del juego ingresado': 'Action Adventure Casual Indie Single-player Downloadable Content',
 'Juegos recomendados': {'Pat &amp; Mat': 'Casual Puzzle Cold War Family Friendly Comedy Funny Illuminati Sexual Content Memes Difficult Story Rich Violent Nudity Single-player Steam Cloud',
  'Streets of Rage': "Action Beat 'em up Classic Arcade 2D Great Soundtrack Retro Local Co-Op Single-player Shared/Split Screen Partial Controller Support",
  'Dreamfall: The Longest Journey': 'Adventure RPG Female Protagonist Story Rich Fantasy Sci-fi Singleplayer Third Person Futuristic Great Soundtrack Atmospheric Cyberpunk Single-player',
  'Helldorado': 'Strategy Western Single-player',
  'Death Track®: Resurrection': 'Racing Action Single-player Steam Cloud'}}

Bueno, me cansé, a partir de los siguientes, voy a probar iterando con un bucle, que considero lo más correcto aunque más tardado también.

In [73]:
# Importo la librería tqdm para ver el progreso del bucle.
from tqdm import tqdm

# Por cada item_id en la la columna de items_id del dataframe de juegos, 
for item_id in tqdm(juegos_steam["item_id"], desc="Procesando juegos"):
    try:
        # Veo si se puede almacenar en una variable local el resultado / salida de la función con ese item_id
        recomendaciones = recomendacion_juego(item_id)
    except Exception as e:
        print(f"Error al obtener recomendaciones para el juego {item_id}: {str(e)}")


Procesando juegos: 100%|██████████| 22530/22530 [2:18:54<00:00,  2.70it/s]  


Bueno, con lo anterior, confirmo que todos los items_id son válidos, por lo cuál doy por finalizada la etapa de prueba de este sistema de recomendación. Lo que voy a hacer a continuación es implementarlo en el main. 